# Coherence Operator Ĉ - Practical Examples

> **📐 Mathematical Theory**: All rigorous mathematical formalization is in **[Mathematical Foundations §3.1](mathematical_foundations.md#31-coherence-operator-ĉ)**
>
> This notebook provides **practical code examples** demonstrating the theory.

## Example 1: Abstract Operator (Mathematics Package)

Using the abstract `CoherenceOperator` from `tnfr.mathematics` to verify spectral properties.

In [ ]:
import numpy as np

from tnfr.mathematics.operators import CoherenceOperator
from tnfr.mathematics.spaces import HilbertSpace

# Create example coherence matrix
coherence_matrix = np.array(
    [
        [0.7, 0.05, 0.0],
        [0.05, 0.9, 0.02],
        [0.0, 0.02, 0.6],
    ],
    dtype=np.complex128,
)

operator = CoherenceOperator(coherence_matrix)
space = HilbertSpace(dimension=3)
state = np.array([0.6, 0.5, 0.2], dtype=np.complex128)
state = state / space.norm(state)

expectation_value = operator.expectation(state)

print("Operator Properties:")
print(f"  Hermitian: {operator.is_hermitian()}")
print(f"  Positive semidefinite: {operator.is_positive_semidefinite()}")
print(f"  c_min: {operator.c_min:.6f}")
print(f"  Expectation ⟨ψ|Ĉ|ψ⟩: {expectation_value:.6f}")

## Example 2: Practical TNFR Network Implementation

Computing the coherence matrix W from actual network nodes using `coherence_matrix()`.

This demonstrates the **implementation bridge** from §3.1.1: how `wᵢⱼ ≈ ⟨i|Ĉ|j⟩` is computed from structural similarities.

In [ ]:
import networkx as nx
from tnfr.metrics.coherence import coherence_matrix
from tnfr.metrics.common import compute_coherence

# Create TNFR network
G = nx.Graph()
G.add_node(0, theta=0.0, EPI=1.0, vf=1.0, Si=0.8)
G.add_node(1, theta=0.5, EPI=1.2, vf=1.1, Si=0.7)
G.add_node(2, theta=1.0, EPI=0.9, vf=1.2, Si=0.9)
G.add_edge(0, 1)
G.add_edge(1, 2)

# Compute coherence matrix W approximating Ĉ
nodes, W = coherence_matrix(G, use_numpy=True)

print(f"Network nodes: {nodes}")
print("\nCoherence matrix W (approximates Ĉ):")

# Convert to dense for display
if isinstance(W, list) and W and isinstance(W[0], tuple):
    W_dense = np.zeros((len(nodes), len(nodes)))
    for i, j, w in W:
        W_dense[i, j] = w
else:
    W_dense = np.array(W)

print(W_dense)
print(f"\nSymmetric (W = W^T): {np.allclose(W_dense, W_dense.T)}")

# Compute spectrum
eigenvalues = np.linalg.eigvalsh(W_dense)
print(f"\nSpectrum σ(Ĉ): {eigenvalues}")

# Total coherence C(t)
for i in range(3):
    G.nodes[i]['dnfr'] = 0.1 * (i + 1)
    G.nodes[i]['dEPI'] = 0.05 * (i + 1)

C_t = compute_coherence(G)
print(f"\nTotal coherence C(t): {C_t:.6f}")

## Interpretation

The practical coherence matrix W provides:
1. **Structural coupling**: `wᵢⱼ` values show how nodes are coherently coupled
2. **Spectral modes**: Eigenvalues reveal coherence "modes" of the network
3. **Global stability**: `C(t)` aggregates network-wide structural stability

This demonstrates TNFR's core principle: **coherence emerges from resonant structural relationships**.

### Further Reading

- **[Mathematical Foundations §3.1](mathematical_foundations.md#31-coherence-operator-ĉ)** - Complete theory
- **[Implementation Bridge §3.1.1](mathematical_foundations.md#311-implementation-bridge-theory-to-code)** - wᵢⱼ construction details
- **[GLOSSARY.md](../../GLOSSARY.md)** - Quick operational reference
- **`src/tnfr/metrics/coherence.py`** - Source implementation